# Streaming1

Run the code for the function `main`
=> Click on it and press `Shift` + `<ENTER>`

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext

def main(timeout, func, window):
  # Create a local StreamingContext and SparkContext.
  # In the databricks environment a default SparkContext already exists so we can't create a second one.
  sc = SparkContext("local[2]", "Streaming-Exercise")
  ssc = StreamingContext(sc, window)
  stream = ssc.socketTextStream("127.0.0.1", 9999)
  
  func(stream)
  try:
    ssc.start()             # Start the computation
    ssc.awaitTerminationOrTimeout(timeout)  # To see the output in Python we have to wait until it's finished.
  except Exception as e:
    print(str(e))
  finally:
    ssc.stop(False)

## Part One

In [2]:
def print_stream(stream):
  # TODO - Print the Stream (one line of code)
  stream.pprint()

In [3]:
# Let it run (start first the netcat Notebook to see some output)
main(10,print_stream,1)

NameError: name 'sc' is not defined

## Part Two

In [ ]:
def count_words(stream):
  # TODO
  # 1) Split the Stream into words
  # 2) Count the words
  # 3) Print them
  stream.flatMap(lambda line: line.split(" "))\
  .map(lambda word: (word, 1))\
  .reduceByKey(lambda x,y: x+y)\
  .foreachRDD(printRDD)
  
def printRDD(rdd):
  print(rdd.collect())

In [ ]:
main(10,count_words,1)

[('Goodbye', 3), ('Bye', 3), ("isn't", 3), ('Spark', 6), ('ass', 3), ('Hello', 6), ('World', 6), ('allowed.', 3)]
[('Goodbye', 3), ('Bye', 3), ("isn't", 3), ('Spark', 6), ('ass', 3), ('Hello', 6), ('World', 6), ('allowed.', 3)]
[('Goodbye', 3), ('Bye', 3), ("isn't", 3), ('Spark', 6), ('ass', 3), ('Hello', 6), ('World', 6), ('allowed.', 3)]
[('Goodbye', 3), ('Bye', 3), ("isn't", 3), ('Spark', 6), ('ass', 3), ('Hello', 6), ('World', 6), ('allowed.', 3)]
[('Goodbye', 3), ('Bye', 3), ("isn't", 3), ('Spark', 6), ('ass', 3), ('Hello', 6), ('World', 6), ('allowed.', 3)]
[('Goodbye', 3), ('Bye', 3), ("isn't", 3), ('Spark', 6), ('ass', 3), ('Hello', 6), ('World', 6), ('allowed.', 3)]
[('Goodbye', 3), ('Bye', 3), ("isn't", 3), ('Spark', 6), ('ass', 3), ('Hello', 6), ('World', 6), ('allowed.', 3)]
[('Goodbye', 3), ('Bye', 3), ("isn't", 3), ('Spark', 6), ('ass', 3), ('Hello', 6), ('World', 6), ('allowed.', 3)]
[('Goodbye', 3), ('Bye', 3), ("isn't", 3), ('Spark', 6), ('ass', 3), ('Hello', 6), ('World', 6), ('allowed.', 3)]
[('Goodbye', 3), ('Bye', 3), ("isn't", 3), ('Spark', 6), ('ass', 3), ('Hello', 6), ('World', 6), ('allowed.', 3)]

## Part Three

In [ ]:
swearwords = set()
escape_char = ["!","%","&","+","$","*","#","="]

def initialize_swearwords():
  global swearwords
  file_url = "/FileStore/tables/swearwords.txt" # Set your own path here
  rdd = sc.textFile(file_url)
  # TODO
  # Fill the swearwords with all words in the file.
  # You should remove the "," and the newline in the file before you fill it in the variable
  import re
  for word in rdd.collect():
    word = re.sub('[,\n]','',word)
    swearwords.add(word)

def random_escape():
  import random
  # TODO
  # Return a random string with escape Character to mask the swearword.
  rep = ""
  length = random.randint(4,8)
  for x in range(0,length):
    rep += random.choice(escape_char)
  return rep

def filter_swearwords(stream):
  # TODO
  # 1) Split the stream into words
  # 2) If the word is a swearword mask it otherwise do nothing
  # 3) Print the text (swearwords masked, other words normal)
  stream.flatMap(lambda text: text.split(" "))\
  .map(lambda word: random_escape() if word in swearwords else word)\
  .foreachRDD(printRdd)
  
def printRdd(rdd):
  print(rdd.collect())
  
initialize_swearwords()


In [ ]:
main(20,filter_swearwords,1)